In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
from utils_fr import *
from inception_blocks import *

In [4]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [5]:
print("Total params: ", FRmodel.count_params())

Total params:  3743280


### THE TRIPLET LOSS

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$
Here, we are using the notation "$[z]_+$" to denote $max(z,0)$.

In [6]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum((anchor - positive)**2, axis=-1)
    
    neg_dist = tf.reduce_sum((anchor - negative)**2, axis=-1)
    
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))
    
    return loss

In [7]:
import tensorflow.compat.v1 as tf1

with tf1.Session() as sess:
    tf1.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf1.random_normal([3, 128], mean=6, stddev=0.1, seed=1),
              tf1.random_normal([3, 128], mean=1, stddev=1, seed=1),
              tf1.random_normal([3, 128], mean=3, stddev=4, seed=1))

    loss = triplet_loss(y_true, y_pred)

    print("loss = " + str(loss.eval()))

loss = 528.1426


In [8]:
FRmodel.compile(optimizer="adam", loss=triplet_loss, metrics=["accuracy"])
load_weights_from_FaceNet(FRmodel)

In [10]:
db = []
db["skuli"] = img_to_encoding("images/skuli.jpg", FRmodel)

ValueError: in user code:

    R:\Work\Anacond\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    R:\Work\Anacond\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    R:\Work\Anacond\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    R:\Work\Anacond\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    R:\Work\Anacond\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    R:\Work\Anacond\lib\site-packages\tensorflow\python\keras\engine\training.py:1445 run_step  **
        outputs = model.predict_step(data)
    R:\